# NLU Testing for Botpress 12
## Now in a Jupyter notebook!

This code is designed to do three things:
  1. It uses your credentials to generate an access token
  2. Takes individual utterances from a test dataset and sends them via converse API to the bot and records the intent
    - If no intent is matched, it will record 'No Intent Matched'
    - After all testing is done it will save the test results
    - Depending on parameters in the .env, extracted slots, confidence, and context are saved
  3. It uses the results to create a confusion matrix that's saved in the base folder. Precision, Recall, and F1 scores are also calculated.

## To use this notebook:
  1. Fill out the "Test Parameters" form with the relevant data
  2. Go to `Runtime` > `Run All`

For the best experience, run this in a Google Colab environment so that all your files are read and written to/from your Google Drive.

In [3]:
#@title Test Parameters { form-width: "50%", display-mode: "both" }
endpoint = "https://endpoint.botpress.cloud" #@param {type:"string"} 
#@markdown _The base URL where botpress is exposed_
user = "user@email.com" #@param {type:"string"}
#@markdown _The usename or email address you use to login to the above instance_
password = "password123" #@param {type:"string"}
#@markdown _The password for the above username_
botId = "myBot" #@param {type:"string"}
#@markdown _The Id of the bot you want to test_
testName = "Test_1" #@param {type:"string"}
#@markdown _A descriptve label for this test_
#
extractEntities = True #@param {type:"boolean"} 
#@markdown _Check to extract entities when running tests_
extractConfidence = False #@param {type:"boolean"}
#@markdown _Check to extract confidence when running tests_
extractContext = False #@param {type:"boolean"}
#@markdown _Check to extract context when running tests_

testPath = "MyDrive/NLU-Testing/Software-Test/" #@param {type:"string"}
#@markdown _Where in your Google Drive your test folder lives. It should start with "MyDrive/"_
testFilename = "Test-Phrases.csv" #@param {type:"string"}
#@markdown _The CSV file that contains your test data. The first two columns **must** be Utterance | Expected_
isGoogleColab = True #@param{type:"boolean"}
#@markdown _True if you're running this notebook in a Colab env and need to pull files from Google Drive_

callsPerSecond = 15 #@param {type:"integer"}
#@markdown _The maximum number of converse API calls per second_

In [4]:
#@title Install dependencies
import requests
import pandas
import os
import json
import re
import uuid
import sys
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
from concurrent.futures import ThreadPoolExecutor, as_completed
%pip install ratelimit
from ratelimit import limits, RateLimitException, sleep_and_retry
from tqdm.notebook import tqdm, trange
if isGoogleColab:
  from google.colab import drive
  drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Definitions
def getToken(endpoint, email, password):
    '''
    Returns a valid authentication token or None in case of an error.

            Parameters:
                    endpoint (string): The exposed Botpress URL
                    email (string): A valid username or email
                    password (string): The password for the supplied username or email.

            Returns:
                    token (string): A JWT token string or None if there was an error.
    '''
    try:
        token = requests.post(f"{endpoint}/api/v1/auth/login/basic/default", data={"email":email, "password":password}).json()["payload"]["jwt"]
        return token
    except:
        print('Error retrieving auth token- check your credentials')
        return None

@sleep_and_retry
@limits(calls=callsPerSecond, period=1)
def getActual(token, utterance, withEntities, withConfidence, withContext):
    '''
    Returns the cleaned intent or qna Id that the bot matched a given training phrase to.

            Parameters:
                    token (string): A valid JWT token.
                    utterance (string): The test phrase to send to the bot.
                    withEntities (boolean): Whether or not to extract & save entities.
                    withConfidence (boolean): Whether or not to extract & save confidence.
                    withContext (boolean): Whether or not to extract & save context.

            Returns:
                    result_dict (dict): A dictionary with the utterance as a key and the returned data in a list as the value.
    '''
    user = uuid.uuid4()
    result_dict = {}
    payload= {
    "type":"text",
    "text":utterance
   # "includedContexts":["hr-bot", "it-bot"]
    }
    auth= {"Authorization": "Bearer "+token}
    url = f"{endpoint}/api/v1/bots/{botId}/converse/{user}/secured?include=decision,nlu"
    response = requests.post(url, data=payload, headers=auth)
    slots = ""
    #try finding confidence in intent.confidence
    try:
        actual = "No Intent Matched" if response.json()["nlu"]["intent"]["name"] == 'none' else response.json()["nlu"]["intent"]["name"]
        extracted = response.json()['nlu']['slots']
        #Add entities if desired
        if((len(extracted)>0) & withEntities):
            slots = [f"{extracted[i]['name']} was extracted from \"{extracted[i]['source']}\" and normalized to {extracted[i]['value']}"
                    for i in extracted]
        else: entities = None
        #Add confidence if desired
        if(withConfidence):
            conf = response.json()["nlu"]["intent"]["confidence"]
        else: conf = None
        if(withContext):
            ctxt = response.json()['nlu']['intent']['context']
        else: ctxt = None
    except:
        #print(response.json()['statusCode'])
        actual = "ERROR"
        slots=[]
        conf=0

    if (re.match(r"__qna__",actual)):
        actual = actual[18:]
    
    result_dict[utterance] = [actual]
    if(withEntities): 
        result_dict[utterance].append(str(slots))
    if(withConfidence): 
        result_dict[utterance].append(np.round(conf*100, 2))
    if(withContext): 
        result_dict[utterance].append(ctxt)
    return result_dict


def runTest(test_df, token):
    '''
    Loop for concurrent converse API calls.

        Parameters:
          token (string): A valid auth token.
          test_df (DataFrame): A DataFrame with two columns containing the test phrases and their labels.
        
        Returns:
          results (List<DataFrame>): A list of DataFrames that contain the results for a given test phrase.
    '''
    threads = []
    results = []   
    with ThreadPoolExecutor(max_workers=10) as executor:
      with tqdm(total=len(test_df.index))as pbar:
        pbar.set_description("Processing...")
        for phrase in test_df.Utterance:
            threads.append(executor.submit(getActual, token, phrase, extractEntities, extractConfidence, extractContext))
        
        for task in as_completed(threads):
            results.append(task.result())
            pbar.update(1)

    return results

def generateConfustionMatrix(labels, yPred, yAct, outputPath, includeValues=True, ):
  '''
    Creates a confusion matrix using matplotlib and also calculates F1 score.

        Parameters:
          labels (ndarray): A list of all possible test labels.
          yPred (List): A list of all the predicted labels.
          yAct (List): A list of all the actual ground-truth lables.
          outputPath (string): Where to save the final chart, not including the filename.
          includeValues (boolean): Whether or not to include numbers in the end confusion matrix.
  '''
  fig, ax = plt.subplots(figsize=(25,20))
  cmp = sklearn.metrics.ConfusionMatrixDisplay.from_predictions(
      yAct, 
      yPred, 
      labels=labels, 
      normalize='true', 
      xticks_rotation='vertical',
      #Set to false for no numbers in the confusiton matrix. useful if you have a lot of intents to test
      include_values=includeValues, 
      ax=ax)

  #Calculate Scores
  f1 = np.around(sklearn.metrics.f1_score(yAct, yPred, labels=labels, average='macro', zero_division=0),4)
  prec = np.around(sklearn.metrics.precision_score(yAct, yPred, labels=labels, average='macro',zero_division=0),4)
  rec = np.around(sklearn.metrics.recall_score(yAct, yPred, labels=labels, average='macro',zero_division=0),4)
  print(f"------{testName} NLU Scoring--------\n Precision Score: {prec} \n Recall Score: {rec} \n F1 Score: {f1}")

  #Add the scores in the plot's title and save the confusion matrix as a PNG
  ax.text(2.5,-1, f"------{testName} NLU Scoring--------\n Precision Score: {prec} \n Recall Score: {rec} \n F1 Score: {f1}",
  size='large')
  fig.tight_layout()
  fig.savefig(outputPath + f"{testName}_confusion_matrix.png", dpi=fig.dpi)


def main():
  '''
    Main function. Run this one!
  '''
  token = getToken(endpoint, user, password)
  if isGoogleColab:
    global testPath    testPath = "/content/drive/"+testPath
  test_df = pandas.read_csv(f"{testPath}{testFilename}",encoding='utf-8', encoding_errors='replace',on_bad_lines='warn')

  if token is None:
    print("Test aborted due to invalid auth token.")
    return

  # Overwrite any previous data with the same labels
  if(testName in test_df.columns):
      test_df.drop(testName, axis=1, inplace=True)
  if( f"{testName}_entities" in test_df.columns):
      test_df.drop(f"{testName}_entities", axis=1, inplace=True)
  if(f"{testName}_confidence" in test_df.columns):
      test_df.drop(f"{testName}_confidence", axis=1, inplace=True)
  if(f"{testName}_context" in test_df.columns):
      test_df.drop(f"{testName}_context", axis=1, inplace=True)

  # Remove any duplicate utterances
  test_df.drop_duplicates(subset='Utterance', keep='last', ignore_index=True, inplace=True)

  # Start the actual test
  print(f"Starting NLU test.... \n {test_df.Utterance.size} utterances detected")
  results = runTest(test_df, token)

  # Use the global params to get the right list of column names
  cols_names = np.ma.masked_array(
    [testName, f"{testName}_entities", f"{testName}_confidence", f"{testName}_context"],
    mask=~np.array([True, extractEntities, extractConfidence, extractContext]))
  
  # Combine all results into a single DataFrame and merge it with the test dataset.
  results_df = pandas.concat(pandas.DataFrame.from_dict(i, orient='index', columns=cols_names[~cols_names.mask].compressed()) for i in results)
  test_df = test_df.merge(right= results_df, how='inner', right_index=True, left_on='Utterance')
  # Repeat the test for any errors, checking twice
  errors = test_df.loc[(test_df[testName]==None)|(test_df[testName]=='')]
  with tqdm(total=len(errors.index))as pbar:
    pbar.set_description("Checking for errors (1 of 2...)")
    for error in errors.Utterance.index:
        test_df.at[error, testName] = list(getActual(test_df.Utterance.iloc[error],extractEntities, extractConfidence, extractContext).values())[0]
        pbar.update(1)
    pbar.close()
  errors = test_df.loc[(test_df[testName]==None)|(test_df[testName]=='')]
  with tqdm(total=len(errors.index))as pbar:
    pbar.set_description("Checking for errors (2 of 2...)")
    for error in errors.Utterance.index:
        test_df.at[error, testName] = list(getActual(test_df.Utterance.iloc[error],extractEntities, extractConfidence, extractContext).values())[0]
        pbar.update(1)
    pbar.close()
  test_df.to_csv(f"{testPath}{testFilename}", index=False)

  generateConfustionMatrix(
      labels = np.sort(test_df[testName].unique()),
      yPred = test_df[testName].tolist(),
      yAct = test_df["Expected"].tolist(),
      outputPath = f"{testPath}",
      includeValues = True)

In [ ]:
#@title Main Code
main()

Error retrieving auth token- check your credentials
Test aborted due to invalid auth token.


b'Skipping line 11: expected 1 fields, saw 3\nSkipping line 14: expected 1 fields, saw 2\nSkipping line 18: expected 1 fields, saw 2\n'
